# Sarcasm Detection using Machine Learning

#### Goal of This File:

##### 1. Import Libraries, Helper Functions, and Constants ==> Data Sourcing and Munging ==> Utility Functions ==> Loading the Data ==> Merging all Individual Files

##### 2. Word Embeddings 

##### 3. Machine Learning Models ==> Small Model Test

##### 4. Conclusion ==> Results

## 1. Import Libraries, Helper Functions and Load Constants

In [111]:
!pip install pandas numpy nltk scikit-learn wordcloud\
seaborn gensim tensorflow imblearn xgboost matplotlib unrar pyunpack more-itertools patool keras-tqdm > /dev/null

The system cannot find the path specified.


In [112]:
VAST = False

if VAST:
    !sudo apt-get install unrar
    !sudo apt-get install rar

'sudo' is not recognized as an internal or external command,
operable program or batch file.
'sudo' is not recognized as an internal or external command,
operable program or batch file.


In [113]:
UTILS_URL = 'https://f005.backblazeb2.com/file/gp-support-files/archived_nlp_ai_utils.py'
UPDATING_VALUES_URL = 'https://f005.backblazeb2.com/file/gp-support-files/archived_updating_values.py'
ALL_LIBS_URL = 'https://f005.backblazeb2.com/file/gp-support-files/archived_all_libs_dl.py'
CHUNKS_URLS_FILE_URL = 'https://f005.backblazeb2.com/file/gp-support-files/chunks_urls.py'

UTILS_FILE_NAME = 'archived_nlp_ai_utils'
UPDATING_VALUES_FILE_NAME = 'archived_updating_values'
ALL_LIBS_FILE_NAME = 'archived_all_libs_dl'
CHUNKS_URLS_FILE_NAME = 'chunks_urls'

DEP_FILE_EXT = '.py'

In [114]:
import requests
import os

def get_dependencies(url,file_name,file_extension):
    if os.path.exists(file_name + file_extension):
        return print(file_name + " already exists.")
    else:
        print(f"downloading {file_name}...")
        r = requests.get(url)
        with open(file_name + file_extension, 'wb') as fd:
            for chunk in r.iter_content():
                fd.write(chunk)

In [115]:
get_dependencies(UTILS_URL,UTILS_FILE_NAME,DEP_FILE_EXT)
get_dependencies(UPDATING_VALUES_URL,UPDATING_VALUES_FILE_NAME,DEP_FILE_EXT)
get_dependencies(ALL_LIBS_URL,ALL_LIBS_FILE_NAME,DEP_FILE_EXT)
get_dependencies(CHUNKS_URLS_FILE_URL,CHUNKS_URLS_FILE_NAME,DEP_FILE_EXT)

archived_nlp_ai_utils already exists.
archived_updating_values already exists.
archived_all_libs_dl already exists.
chunks_urls already exists.


In [116]:
from archived_nlp_ai_utils import *
from chunks_urls import SD_CHUNKS_URLS
from archived_updating_values import *

In [117]:
TF_ENABLE_ONEDNN_OPTS = 0
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
DATA_URLS = SD_CHUNKS_URLS
GLOVE_URL = 'https://f005.backblazeb2.com/file/glove-embeddings-dims/glove.6B.100d.txt'
LIMIT = 10
TRAINED_MODELS_COUNT = TRAINED_MODELS
DATASET_PATH = "../datasets"
PICKLES_DIR = "../sd_pickle_files"
PREPROCESSED_CHUNKS_PATH = DATASET_PATH + "/preprocessed_sd_data_chunks/"
BASE_FILE_NAME = "sd_chunk_"
FILE_FORMAT = ".csv"
DATA_IN_CHUNK = 99744
if LIMIT == 10:
    ACTUAL_DATA_SHAPE = DATA_IN_CHUNK * LIMIT - 2
else:
    ACTUAL_DATA_SHAPE = DATA_IN_CHUNK * LIMIT
RANDOM_STATE = CONST_RANDOM_STATE
READY_DATASET_PATH = "../larger_dataset/ready_for_models/"

### 1.1 Data Sourcing and Munging

#### 1.1.1 Loading The Data

In [118]:
DATASET_PATH

'../datasets'

In [119]:
PREPROCESSED_CHUNKS_PATH

'../datasets/preprocessed_sd_data_chunks/'

In [120]:
if not os.path.exists(DATASET_PATH):
    os.mkdir(DATASET_PATH)
if not os.path.exists(PREPROCESSED_CHUNKS_PATH):
    os.mkdir(PREPROCESSED_CHUNKS_PATH)
get_chunks(DATA_URLS,LIMIT,1,BASE_FILE_NAME,PREPROCESSED_CHUNKS_PATH,FILE_FORMAT)

sd_chunk_1 already exists.
sd_chunk_2 already exists.
sd_chunk_3 already exists.
sd_chunk_4 already exists.
sd_chunk_5 already exists.
sd_chunk_6 already exists.
sd_chunk_7 already exists.
sd_chunk_8 already exists.
sd_chunk_9 already exists.
sd_chunk_10 already exists.


#### 1.1.2 Merging all Individual Files

In [121]:
#get all names of downloaded files
all_file_names = get_all_file_names(BASE_FILE_NAME,LIMIT)

In [122]:
#read all chunks into a list
list_dfs = read_chunks(all_file_names,PREPROCESSED_CHUNKS_PATH,FILE_FORMAT)

In [123]:
#concatenate all chunks into a singular df
df = group_up_chunks(list_dfs)

In [124]:
#check how much of the data was actually downloaded
percent_loaded = check_no_missing_data(df.shape[0],ACTUAL_DATA_SHAPE)
percent_loaded

100.0

In [125]:
df.reset_index(inplace = True)
df.drop(['index'],axis = 1,inplace = True)

In [126]:
X = df['text']
y = df['labels']

In [127]:
X = pd.Series([str(text) for text in X])

y = pd.Series([int(label) for label in y])

In [130]:
if not os.path.exists(PICKLES_DIR):
    os.mkdir(PICKLES_DIR)

## 2. Word Embeddings

### 2.1 Word2Vec

In [131]:
reviews_tokens = [nltk.word_tokenize(text) for text in X]

In [132]:
model = Word2Vec(sentences=reviews_tokens, vector_size=100, window=5, min_count=1, workers=4)

In [133]:
X_W2V = np.zeros((len(reviews_tokens), model.vector_size))
for i, review in enumerate(reviews_tokens):
    for word in review:
        if word in model.wv:
            X_W2V[i] += model.wv[word]
    X_W2V[i] /= len(reviews_tokens)

### 2.2 TF-IDF

### 2.3 FastText

## 3. Machine Learning Models

### 3.1 Word2Vec Train-Validation-Test Split

In [ ]:
TRAIN_PERCENT = 0.8
VALID_TEST_PERCENT = 0.1
TRAIN_SIZE = int(ACTUAL_DATA_SHAPE * TRAIN_PERCENT)
VALID_TEST_SIZE = int(ACTUAL_DATA_SHAPE * VALID_TEST_PERCENT)
TOTAL_TEST_SIZE = TRAIN_SIZE + VALID_TEST_SIZE
train_set = X_W2V[:TRAIN_SIZE]
train_labels = y[:TRAIN_SIZE]
validation_set = X_W2V[TRAIN_SIZE:TOTAL_TEST_SIZE]
validation_labels = y[TRAIN_SIZE:TOTAL_TEST_SIZE]
test_set = X_W2V[TOTAL_TEST_SIZE:]
test_labels = y[TOTAL_TEST_SIZE:]

### 3.2 TF-IDF Train-Validation-Test Split

### 3.3 FastText Train-Validation-Test Split

### 3.4 Naive Bayes

### 3.5 SVM

### 3.6 Random Forest

In [ ]:
rf_clf_w2v = RandomForestClassifier(random_state=RANDOM_STATE)
rf_clf_w2v.fit(train_set, train_labels)
validation_predictions = rf_clf_w2v.predict(validation_set)
accuracy = metrics.accuracy_score(validation_labels,validation_predictions)

### 3.4 XGBoost

### 3.5 Small Model Test

In [ ]:
test_review = "I totally would love to lend some money to someone I don't know at all!"
test_token = nltk.word_tokenize(test_review)
test_vector = np.zeros(model.vector_size)
for word in test_token:
    if word in model.wv:
        test_vector += model.wv[word]
test_vector /= len(test_token)
predicted_label = clf.predict([test_vector])[0]

## 4. Conclusion

### 4.1 Results